#Lista 1
-------------


Objetivo: treinar um modelo de classificação utilizando o modelo de regressão Logística e árvore de decisão.


Banco utilizado: Detecção de doenças no fígado.


[Link](https://drive.google.com/file/d/1jnLwuv4e_ZeCqluMXs3Mar2TGetumtVv/view?usp=drive_link) para o banco


Descrição do banco:
> Este banco contém registro de 416 pacientes diagnosticados com doença no fígado e 167 pacientes sem. Esta informação está na coluna Selector.
>
> Há 10 variáveis no banco:
>
> * age: idade
> * Gender: gênero do paciente
> * TB: Bilirrubina total
> * DB: Bilirrubina direta
> * Alkphos: fosfatase alcalina.
> * Sgpt: transaminase glutâmico-pirúvica sérica (TGP)
> * Sgot: transaminase glutâmico-oxalacética sérica (TGO)
> * TP: Proteína total
> * ALB: Albumina
> * A/G Ratio: Relação Albumina:Globulina


Exercício:


1. Carregue o banco de dados e analise suas features. Transforme a feature sexo em uma variavel dummy.
2. Separe o banco em 80% para treino e 20% para teste.
3. Treine um modelo de regressão logística.
  3.1 Interprete dois coeficientes.
4. Treine um modelo de árvore de decisão. Utilize max_depth de 4.
  4.1 Quais são as duas features mais importantes?
5. Construa um relatório dos dois modelos.
  5.1 Tomando como decisão a precisão para detectar doença no fígado qual é o melhor modelo?


 5.2 Tomando como decisão o f1 score do rótulo de pacientes com doença no fígado qual é o melhor modelo?


Obs.: Utilize o **mesmo** banco de treino e de teste para construir e avaliar os dois modelos.

##1. Carga do banco
--------------------

In [252]:
import pandas as pd
import numpy as np
df = pd.read_csv('Indian Liver Patient Dataset (ILPD).csv')
df.head()

In [253]:
df.info()

In [254]:
df.isna().sum()

In [255]:
df = df.dropna()

In [256]:
df['Selector'].value_counts()

In [257]:
# transformando selector em 0 e 1   
df['Selector'] = df['Selector'].map({1:0, 2:1})

In [258]:
# dummies sexo
df = pd.get_dummies(df , columns=['Gender'], drop_first=True)

In [259]:
df.head()

##2. Quebra do banco
--------------------

In [260]:
from sklearn.model_selection import train_test_split
X = df.drop('Selector', axis=1)
Y = df['Selector']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=15)

##3. Primeiro modelo
--------------------

In [261]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='none')
model.fit(X_train, Y_train)

In [262]:
pd.DataFrame(model.coef_.T, index=X_train.columns) # coeficientes do modelo de regressão logística 

##3.1 Interpretação de dois coeficientes
--------------------

In [263]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, RocCurveDisplay
Y_pred = model.predict(X_test)
print(classification_report(Y_test, Y_pred))

In [264]:
confusion_matrix(Y_test, Y_pred)

In [265]:
fpr, tpr, thresholds = roc_curve(Y_test, Y_pred)
roc_auc = roc_auc_score(Y_test, Y_pred)
display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc)
display.plot()

In [266]:
from sklearn.metrics import accuracy_score,precision_score,f1_score
import plotly.express as px

predict_proba = model.predict_proba(X_test)[:, 1]

f1_lista = []
precision_lista = []
for thr in np.arange(0, 1, .01):
    f1_lista.append(f1_score(Y_test, predict_proba >= thr))
    precision_lista.append(precision_score(Y_test, predict_proba >= thr))


In [267]:
px.line(x=np.arange(0, 1, .01), y=f1_lista, labels={'x':'Threshold', 'y':'F1 Score'})

In [268]:
print(classification_report(Y_test, predict_proba >= .31))

In [269]:
px.line(x=np.arange(0, 1, .01), y=precision_lista, labels={'x':'Threshold', 'y':'Precision'})

In [270]:
print(classification_report(Y_test, predict_proba >= .49))

##4. Segundo modelo
--------------------

In [271]:
# árvore de decisão 
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=4,random_state=15)
model.fit(X_train, Y_train)

##4.1 Duas features mais importantes
--------------------

In [272]:
pd.DataFrame(model.feature_importances_, index=X_train.columns, columns=['importance']).sort_values(by='importance', ascending=False).head(2)

##5. Avaliação do modelo
--------------------

In [273]:
Y_pred = model.predict(X_test)
print(classification_report(Y_test, Y_pred))

In [274]:
predict_proba = model.predict_proba(X_test)
predict_proba

In [275]:

predict_proba = model.predict_proba(X_test)[:, 1]

f1_lista = []
precision_lista = []
for thr in np.arange(0, 1, .01):
    f1_lista.append(f1_score(Y_test, predict_proba>=thr))
    precision_lista.append(precision_score(Y_test, predict_proba>=thr))



In [276]:
px.line(x=np.arange(0, 1, .01), y=precision_lista, labels={'x':'Threshold', 'y':'Precision'})

In [277]:
print(classification_report(Y_test, predict_proba >= .73))

In [278]:
px.line(x=np.arange(0, 1, .01), y=f1_lista, labels={'x':'Threshold', 'y':'F1 Score'})

In [279]:
print(classification_report(Y_test, predict_proba >= .15))

##5.1 Qual modelo escolhido olhando para a precisão de detecção de doença no fígado?
--------------------
Olhando para os resultados ótimos dos modelos:
- Regressão Logística: Threshold de 0.49
- Árvore de Decisão: Threshold de 0.73
- A precisão da Regressão Logística é de 0.55 e da Árvore de Decisão é de 0.75
- O modelo escolhido é a Árvore de Decisão

##5.2 Qual modelo escolhido olhando para o f1-score do rótulo de doença do fígado?
--------------------
Olhando para os resultados ótimos dos modelos onde 1 é o rótulo de doença no fígado:
- Regressão Logística: Threshold de 0.31
- Árvore de Decisão: Threshold de 0.15
- O f1-score da Regressão Logística é de 0.57 e da Árvore de Decisão é de 0.49
- O modelo escolhido é a Regressão Logística